#### Genetic Programming: 4-parity problem with ADFs (based on DEAP example)
CE310 Evolutionary Computation and Genetic Programming<br>
Reinhold Scherer, Spring 2021, Unit 7

In [1]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import random
import operator
import math

import numpy

from deap import base
from deap import creator
from deap import gp
from deap import tools


# ====================================================================
# Initialize Parity problem input and output matrices
N=4
inputs  = [None] * 2**4
outputs = [None] * 2**4


for i in range(2**4):
    row=format(i, '04b')
    outputs[i] = row.count('1')%2==0

    inputs[i] = [None] * 4
    for j in range(N):
        inputs[i][j] = row[j]=="1"
        
print(inputs)
print(outputs)

# ====================================================================    
# Define new functions
def NAND(x,y):
    return not (x and y)

def NOR(x,y):
    return not (x or y)


adfset1 = gp.PrimitiveSet("ADF1", 2)
adfset1.addPrimitive(operator.and_, 2)
adfset1.addPrimitive(operator.or_, 2)
adfset1.addPrimitive(NAND, 2)
adfset1.addPrimitive(NOR, 2)

adfset0 = gp.PrimitiveSet("ADF0", 2)
adfset0.addPrimitive(operator.and_, 2)
adfset0.addPrimitive(operator.or_, 2)
adfset0.addPrimitive(NAND, 2)
adfset0.addPrimitive(NOR, 2)
adfset0.addADF(adfset1)

pset = gp.PrimitiveSet("MAIN", 4)
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(NAND, 2)
pset.addPrimitive(NOR, 2)
pset.addADF(adfset0)
pset.addADF(adfset1)


psets = (pset, adfset0, adfset1)

creator.create("FitnessMin", base.Fitness, weights=(+1.0,))
creator.create("Tree", gp.PrimitiveTree)

creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('adf_expr0', gp.genFull, pset=adfset0, min_=2, max_=3)
toolbox.register('adf_expr1', gp.genFull, pset=adfset1, min_=2, max_=3)
toolbox.register('main_expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=4)

toolbox.register('ADF0', tools.initIterate, creator.Tree, toolbox.adf_expr0)
toolbox.register('ADF1', tools.initIterate, creator.Tree, toolbox.adf_expr1)
toolbox.register('MAIN', tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN, toolbox.ADF0, toolbox.ADF1]

toolbox.register('individual', tools.initCycle, creator.Individual, func_cycle)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(individual)
    return -sum(abs(func(*in_) - out) for in_, out in zip(inputs, outputs)),



toolbox.register('compile', gp.compileADF, psets=psets)
toolbox.register('evaluate', evalSymbReg)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr', gp.genFull, min_=1, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr)

#toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=64))
#toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=64))

random.seed() #(1024)
ind = toolbox.individual()
    
pop = toolbox.population(n=4000)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)
    
logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max"
    
CXPB, MUTPB, NGEN = 0.7, 0.00, 30
   
# Evaluate the entire population
for ind in pop:
    ind.fitness.values = toolbox.evaluate(ind)

hof.update(pop)
record = stats.compile(pop)
logbook.record(gen=0, evals=len(pop), **record)
print(logbook.stream)
    
for g in range(1, NGEN):
    # Select the offspring
    offspring = toolbox.select(pop, len(pop))
    # Clone the offspring
    offspring = [toolbox.clone(ind) for ind in offspring]
    
    # Apply crossover and mutation
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        for tree1, tree2 in zip(ind1, ind2):
            if random.random() < CXPB:
                toolbox.mate(tree1, tree2)
                del ind1.fitness.values
                del ind2.fitness.values

    for ind in offspring:
        for tree, pset in zip(ind, psets):
            if random.random() < MUTPB:
                toolbox.mutate(individual=tree, pset=pset)
                del ind.fitness.values
                            
    # Evaluate the individuals with an invalid fitness
    invalids = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalids:
        ind.fitness.values = toolbox.evaluate(ind)
                
    # Replacement of the population by the offspring
    pop = offspring
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals=len(invalids), **record)
    print(logbook.stream)
    
print('Best individual : ', hof[0][0], hof[0].fitness)



[[False, False, False, False], [False, False, False, True], [False, False, True, False], [False, False, True, True], [False, True, False, False], [False, True, False, True], [False, True, True, False], [False, True, True, True], [True, False, False, False], [True, False, False, True], [True, False, True, False], [True, False, True, True], [True, True, False, False], [True, True, False, True], [True, True, True, False], [True, True, True, True]]
[True, False, False, True, False, True, True, False, False, True, True, False, True, False, False, True]
gen	evals	std     	min	avg     	max
0  	4000 	0.280622	-11	-7.99875	-4 
1  	3902 	0.286768	-10	-7.958  	-6 
2  	3898 	0.342507	-10	-7.92625	-6 
3  	3894 	0.411302	-10	-7.87925	-5 
4  	3896 	0.498287	-10	-7.80775	-4 
5  	3906 	0.581132	-12	-7.70925	-5 
6  	3890 	0.625977	-10	-7.59825	-5 
7  	3890 	0.665359	-10	-7.47175	-5 
8  	3860 	0.725437	-10	-7.329  	-4 
9  	3884 	0.783772	-11	-7.17875	-4 
10 	3900 	0.8469  	-12	-7.00325	-4 
11 	3882 	0.88

In [2]:
print(hof[0][0])
print(hof[0][1])
print(hof[0][2])

ADF1(or_(NOR(or_(and_(ARG3, ARG3), ARG3), NOR(ARG1, ADF1(ARG3, ARG1))), ADF1(ADF0(ARG2, ARG2), NAND(ARG2, ARG1))), ADF1(ADF0(NAND(ARG0, ARG2), and_(ARG3, ARG3)), ADF1(and_(ARG1, ARG2), ADF1(ARG1, ARG0))))
or_(or_(NOR(NAND(ARG1, ARG1), and_(ARG1, ARG1)), ARG1), ARG1)
and_(or_(NAND(ARG0, and_(or_(ARG0, ARG1), ARG1)), NOR(ARG1, ARG1)), NAND(NOR(or_(ARG0, ARG1), ARG1), NOR(ARG1, ARG0)))


In [3]:
f = toolbox.compile(expr=hof[0])

for in_ in inputs:
    print(f(*in_))
    
print(outputs)

True
False
False
True
False
True
True
False
False
True
True
False
True
False
False
True
[True, False, False, True, False, True, True, False, False, True, True, False, True, False, False, True]
